# Conversation War between LLMs!!

This code sets up a conversation between GPT(Connected via API) and llama3.2 (local) with different tones for both

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import ollama

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

In [ ]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

In [ ]:
!ollama pull llama3.2

In [ ]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
ollama_model = "llama3.2"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

ollama_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
ollama_messages = ["Hi"]

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, ollama in zip(gpt_messages, ollama_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": ollama})
    # print(messages)
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [ ]:
def call_ollama():
    messages = [{"role": "system", "content": ollama_system}]
    for gpt, ollama in zip(gpt_messages, ollama_messages):
        messages.append({"role": "assistant", "content": ollama})
        messages.append({"role": "user", "content": gpt})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    # print(messages)
    completion = ollama_via_openai.chat.completions.create(
        model=ollama_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
gpt_messages = ["Hi there"]
ollama_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Ollama:\n{ollama_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:")
    display(Markdown(gpt_next))
    print(f"\n")
    gpt_messages.append(gpt_next)
    
    ollama_next = call_ollama()
    print(f"Ollama:")
    display(Markdown(ollama_next))
    print(f"\n")
    ollama_messages.append(ollama_next)